## Q-Table learning

In [2]:
import gym
import numpy as np
import time

### Load environment

In [3]:
env = gym.make('FrozenLake-v0')

### Implement Q-Table learning algorithm

### Temporal difference: (metoda różnic czasowych)
$$
Q(s_t,a_t) = Q(s_t,a_t) + \alpha (r + \gamma \max_{a} Q(s_{t+1}, a) - Q(s_t,a_t))
$$
where<br>
$ \alpha $ - step size (learning rate)<br>
$ \gamma $ - discount factor<br>
$ s_t $ - current state<br>
$ s_{t+1} $ - next state <br>
$ r $ - reward<br>
$ a $ - action

In [14]:
#Initialize table with all zeros
Q = np.zeros([env.observation_space.n, env.action_space.n])
#size of Q table
print(Q.shape)

(64, 4)


### Simple example

In [6]:
a = .8 #alpha
y = .95 #gamma
Q = np.zeros([env.observation_space.n, env.action_space.n])
#reset environment, state = 0
state = env.reset()
    
reward = 0
done = False
for i in range(100):
        
    #choose action by greedily (with noise) picking from Q table (max value of current state)
    action = np.argmax(Q[state,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))

    #get new state and reward from environment
    next_state, reward, done, _ = env.step(action)
        
    #update Q-Table with new knowledge
    Q[state, action] = Q[state, action] + a*(reward + y*np.max(Q[next_state,:]) - Q[state, action])
        
    #update current state
    state = next_state
       
    if done == True:
        break

env.render()

  (Down)
SFFF
FHFH
FFFH
HFFG


### Example - one episode:

In [9]:
a = .8 #alpha
y = .95 #gamma
Q = np.zeros([env.observation_space.n, env.action_space.n])
num_episodes = 1

for i in range(num_episodes):
    #reset environment, state = 0
    state = env.reset()
    
    reward = 0
    done = False
    total_reward = 0
    visited_states = [0, ]
    choosed_actions = []
    for j in range(100):
        
        #choose action by greedily (with noise) picking from Q table (max value of current state)
        action = np.argmax(Q[state,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        
        #get new state and reward from environment
        next_state, reward, done, _ = env.step(action)
        
        #update Q-Table with new knowledge
        Q[state, action] = Q[state, action] + a*(reward + y*np.max(Q[next_state,:]) - Q[state, action])
        
        #update current state
        state = next_state
        
        visited_states.append(state)
        choosed_actions.append(
        {
            0 : 'l',
            1 : 'd',
            2 : 'r',
            3 : 'u'
        }[action])
        total_reward += reward        
        
        if done == True:
            break

choosed_actions.append('-')
print('Visited states and actions:')
print(np.array([visited_states, choosed_actions]))
print()
print('Last move:')
env.render()
print()
print('Numbers representing states:')
print(np.arange(0,16).reshape(4,4))
print()
print('Total reward: ',total_reward)

Visited states and actions:
[['0' '0' '0' '0' '0' '0' '0' '4' '0' '4' '0' '0' '4' '5']
 ['d' 'u' 'l' 'u' 'd' 'l' 'd' 'l' 'l' 'l' 'd' 'd' 'u' '-']]

Last move:
  (Up)
SFFF
FHFH
FFFH
HFFG

Numbers representing states:
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]

Total reward:  0.0


We add noise when choosing action, because in this environment we don't have penalties for jumping into the hole (H state). Without noise the agent would always choose to go left (np.argmax() returns first occurance of the max value, so at the beginning when all values for states are zeros [0, 0, 0, 0], it returns index 0 all the time - which corresponds to going left).

### Example - 2000 episodes:

In [4]:
a = .8 #alpha
y = .95 #gamma
Q = np.zeros([env.observation_space.n, env.action_space.n])
num_episodes = 2000
rList = [] #reward list
for i in range(num_episodes):
    state = env.reset()
    
    reward = 0
    done = False
    total_reward = 0
    for j in range(100):
        action = np.argmax(Q[state,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        next_state, reward, done, _ = env.step(action)
        Q[state, action] = Q[state, action] + a*(reward + y*np.max(Q[next_state,:]) - Q[state, action])
        
        state = next_state
        
        total_reward += reward        
        
        if done == True:
            break
    rList.append(total_reward)
    
print("Score over time: " +  str(sum(rList)/num_episodes))
print(Q)

Score over time: 0.6215
[[  2.68878671e-01   7.39041589e-03   1.28184098e-02   1.04011804e-02]
 [  1.73668520e-05   2.84134126e-04   1.21605115e-04   1.46316457e-01]
 [  1.91046158e-03   1.91429324e-01   2.58617449e-03   9.99798664e-03]
 [  6.64379324e-04   0.00000000e+00   1.71136374e-03   1.15316541e-01]
 [  4.02649739e-01   1.26964469e-03   9.52508920e-06   2.56476648e-03]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  4.92853060e-03   7.28023995e-07   3.44984908e-07   4.03377905e-07]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.43235312e-03   2.23138527e-04   1.06093977e-03   2.64026144e-01]
 [  1.70353475e-03   6.27456309e-01   1.29758701e-03   7.39328000e-04]
 [  6.55900974e-01   5.75034450e-04   1.94560000e-04   3.53195071e-04]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.73746979e-04   3.36307716e-03   3.20879399e-01 

## Numpy stuff
### What Q[state, :] means?

**Python slice notation**<br>
Given an array "a":
```python
a[start:end] # items start through end-1
a[start:]    # items start through the rest of the array
a[:end]      # items from the beginning through end-1
a[:]         # a copy of the whole array

a[start:end:step] # start through not past end, by step
```

Let's consider we have a matrices like this:

In [57]:
A = np.array([[1, 1, 1],[2, 2, 2],[3, 3, 3]])
B = np.array([[1, 2, 3],[1, 2, 3],[1, 2, 3]])
print(A)
print()
print(B)

[[1 1 1]
 [2 2 2]
 [3 3 3]]

[[1 2 3]
 [1 2 3]
 [1 2 3]]


Getting a "slice" of array means to get subarray.

In [61]:
#get first row
subA = A[0, :]
#first argument is row number - 1st row (0 indexed array)
#second argument is column numbers - ":" means every column
print(subA)

[1 1 1]


In [62]:
#get first two columns
subB = B[:, 0:2]
# 0:2 - means column [0, 2) - column 0 to 2, without 2
print(subB)

[[1 2]
 [1 2]
 [1 2]]


### np.argmax(a, axis=None, out=None)
Returns the indices of the maximum values along an axis.

In [70]:
array = np.array([[1,3,2],[3,4,16]])
print(array)

[[ 1  3  2]
 [ 3  4 16]]


In [71]:
idx = np.argmax(array[0,:])
print(array[0,:])
print('max value in first row on index: ', idx)
print()
idx = np.argmax(array[1,:])
print(array[1,:])
print('max value in second row on index: ', idx)

[1 3 2]
max value in first row on index:  1

[ 3  4 16]
max value in second row on index:  2
